In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# !rm multinli_1.0.zip

In [3]:
from glob import glob

files = sorted(glob('multinli_1.0/*.jsonl'))
files

['multinli_1.0/multinli_1.0_dev_matched.jsonl',
 'multinli_1.0/multinli_1.0_dev_mismatched.jsonl',
 'multinli_1.0/multinli_1.0_train.jsonl']

In [4]:
import json

data = []
with open(files[-1]) as fopen:
    for l in fopen:
        data.append(json.loads(l))

len(data)

392702

In [5]:
data[0]

{'annotator_labels': ['neutral'],
 'genre': 'government',
 'gold_label': 'neutral',
 'pairID': '31193n',
 'promptID': '31193',
 'sentence1': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'sentence1_binary_parse': '( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )',
 'sentence1_parse': '(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))',
 'sentence2': 'Product and geography are what make cream skimming work. ',
 'sentence2_binary_parse': '( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )',
 'sentence2_parse': '(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))'}

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('mesolitica/finetune-translation-t5-small-standard-bahasa-cased')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/finetune-translation-t5-small-standard-bahasa-cased')

In [7]:
_ = model.cuda()

In [11]:
%%time

batch = [data[0]['sentence1'], data[0]['sentence2']]
t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
inputs = tokenizer(t, return_tensors="pt", padding = True)
for k in inputs.keys():
    inputs[k] = inputs[k].cuda()
translated_tokens = model.generate(**inputs, max_length=500)
decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

CPU times: user 94.3 ms, sys: 1.28 ms, total: 95.6 ms
Wall time: 94 ms


In [10]:
import torch

torch.cuda.empty_cache()

In [12]:
from tqdm import tqdm

with open('translated-mnli-train.jsonl', 'w') as fopenl:
    for d in tqdm(data):
        torch.cuda.empty_cache()
        batch = [d['sentence1'], d['sentence2']]
        t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
        inputs = tokenizer(t, return_tensors="pt", padding = True)
        for k in inputs.keys():
            inputs[k] = inputs[k].cuda()
        translated_tokens = model.generate(**inputs, max_length=500)
        decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        
        d['translate'] = decoded
        fopenl.write(f'{json.dumps(d)}\n')

 14%|████████████▌                                                                             | 54567/392702 [2:20:20<42:11:06,  2.23it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 392702/392702 [16:33:54<00:00,  6.59it/s]
